In [2]:
import os
os.chdir(os.path.join(os.getcwd(), '..','..', '..'))

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
import csv
import collections
import numpy as np
import pandas as pd
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.filters import Filter
from weka.attribute_selection import ASEvaluation, AttributeSelection
from weka.classifiers import Classifier, Evaluation
from imblearn.over_sampling import SMOTE

jvm.start()

In [5]:
class KNN:
    def __init__(self, x_train, y_train, x_test, y_test, n, k):
        self.knn = KNeighborsClassifier(n_neighbors=n)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k

    def feature_selection_SelectKBest(self):
        k_best = SelectKBest(chi2, k=self.k)
        k_best.fit(self.x_train, self.y_train)
        self.x_train = k_best.transform(self.x_train)
        self.x_test = k_best.transform(self.x_test)
        
    def feature_selection_ExtraTreesClassifier(self):
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(self.x_train, self.y_train)
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]
        self.x_train = self.x_train[:, indices[:self.k]]
        self.x_test = self.x_test[:, indices[:self.k]]
    
    def feature_selection_InfoGainAttributeEval(self, morbidity):
        loader = Loader(classname="weka.core.converters.ArffLoader")
        train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
        train_data.class_is_last()

        # Initialize attribute selection
        eval = ASEvaluation(classname="weka.attributeSelection.InfoGainAttributeEval")
        search = AttributeSelection()
        search.evaluator = eval
        search.select_attributes(train_data)
        selected_attributes = search.selected_attributes
        filtered_attributes = np.delete(selected_attributes, [-1])
        # print("Selected attributes:", type(filtered_attributes), filtered_attributes.shape)

        # Apply selected attributes to the training and testing sets
        self.x_train = self.x_train[:, filtered_attributes]
        self.x_test = self.x_test[:, filtered_attributes]

    def train(self):
        self.knn.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.knn.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [6]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']
column_headings = ["Morbidity Class", "KNN1_Macro F1", "KNN1_Micro F1", "KNN5_Macro F1", "KNN5_Micro F1"]

In [7]:
with open("./results/tf-idf-features/performance_KNN_AllFeatures.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
    
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)
    
    else:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
        
    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_AllFeatures.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_AllFeatures.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
For n=1, Macro F1 score: 0.6658190132278012 and Micro F1 Score 0.7002475247524752
For n=5, Macro F1 score: 0.5729768592199544 and Micro F1 Score 0.6335346534653465
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
For n=1, Macro F1 score: 0.6723177144635537 and Micro F1 Score 0.6907692307692308
For n=5, Macro F1 score: 0.6670987946606427 and Micro F1 Score 0.6846153846153846
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
For n=1, Macro F1 score: 0.6556393888462139 and Micro F1 Score 0.6891304347826087
For n=5, Macro F1 score: 0.5736738476837866 and Micro F1 Score 0.6304347826086956
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
For n=1, Macro F1 score: 0.6056469743024839 and Micro F1 Score 0.6502215189873418
For n=5, Macro F1 score: 0.50895788

In [8]:
with open("./results/tf-idf-features/performance_KNN_SelectKBest.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled

        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
            knn1_obj.feature_selection_SelectKBest()
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
            knn5_obj.feature_selection_SelectKBest()
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)
    else:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
    
    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_SelectKBest.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_SelectKBest.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
For n=1, Macro F1 score: 0.8904707524271981 and Micro F1 Score 0.8924752475247526
For n=5, Macro F1 score: 0.8061507492920971 and Micro F1 Score 0.8147623762376238
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
For n=1, Macro F1 score: 0.7220770636753526 and Micro F1 Score 0.7292307692307693
For n=5, Macro F1 score: 0.7329826851655336 and Micro F1 Score 0.7430769230769231
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
For n=1, Macro F1 score: 0.8142757833874731 and Micro F1 Score 0.8206521739130433
For n=5, Macro F1 score: 0.7428792763363237 and Micro F1 Score 0.7554347826086956
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
For n=1, Macro F1 score: 0.7585019159190531 and Micro F1 Score 0.7690822784810127
For n=5, Macro F1 score: 0.67614928

In [9]:
with open("./results/tf-idf-features/performance_KNN_ExtraTreesClassifier.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled

        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
            knn1_obj.feature_selection_ExtraTreesClassifier()
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
            knn5_obj.feature_selection_ExtraTreesClassifier()
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)
    else:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
        
    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_ExtraTreesClassifier.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_ExtraTreesClassifier.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
For n=1, Macro F1 score: 0.8870840405612368 and Micro F1 Score 0.8895049504950494
For n=5, Macro F1 score: 0.8177542748865863 and Micro F1 Score 0.8237623762376238
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
For n=1, Macro F1 score: 0.7517687690974972 and Micro F1 Score 0.7584615384615384
For n=5, Macro F1 score: 0.736961327067114 and Micro F1 Score 0.7461538461538462
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
For n=1, Macro F1 score: 0.8132273195907731 and Micro F1 Score 0.8195652173913043
For n=5, Macro F1 score: 0.7303824101373435 and Micro F1 Score 0.7456521739130435
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
For n=1, Macro F1 score: 0.7492651519124427 and Micro F1 Score 0.7600949367088607
For n=5, Macro F1 score: 0.688755013

In [9]:
with open("./results/tf-idf-features/performance_KNN_InfoGain.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro1 = 1
        f1_micro1 = 1
        f1_macro5 = 1
        f1_micro5 = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        f1_macro_list1 = []
        f1_micro_list1 = []
        f1_macro_list5 = []
        f1_micro_list5 = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training KNN using TF-IDF Representation
            knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
            knn1_obj.feature_selection_InfoGainAttributeEval(morbidity)
            knn1_obj.train()

            f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

            f1_macro_list1.append(f1_macro1)
            f1_micro_list1.append(f1_micro1)

            knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
            knn5_obj.feature_selection_InfoGainAttributeEval(morbidity)
            knn5_obj.train()

            f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

            f1_macro_list5.append(f1_macro5)
            f1_micro_list5.append(f1_micro5)

        f1_macro1 = np.mean(f1_macro_list1)
        f1_micro1 = np.mean(f1_micro_list1)
        f1_macro5 = np.mean(f1_macro_list5)
        f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_InfoGain.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_InfoGain.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.9094597195218244 and Micro F1 Score 0.9650030248033878
For n=5, Macro F1 score: 0.8476552107167418 and Micro F1 Score 0.9475196612220204
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.824406325072748 and Micro F1 Score 0.8284511784511785
For n=5, Macro F1 score: 0.8511751737633242 and Micro F1 Score 0.8556565656565656
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1 and Micro F1 Score 1
For n=5, Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.6887780746824912 and Micro F1 Score 0.7990648743424897
For n=5, Macro F1 score: 0.7203267817584003 and Micro F1 Score 0.8505260081823496
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.8737822893538032 and Micro F1 Score 0.8993734335839599
For n=5, Macro F1 score: 0.862032564768108 and Micro F1 Score 0.883